In [ ]:
import csv

from itertools import *
from functools import *
from collections import defaultdict

import rep
import rep.data

import sklearn
import sklearn.feature_extraction
import sklearn.decomposition

import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt
%matplotlib inline

from overloading import overload

In [ ]:
DATA_PREFIX = 'data/'

# Read data

In [ ]:
import os
os.chdir('{}/{}'.format(os.getcwd(),
                        DATA_PREFIX))

In [ ]:
train_likes_file = open('train_likes.csv')
reader = csv.DictReader(train_likes_file)
train_likes = map(lambda row: (row['user_id'], row['item_id']), reader)

In [ ]:
list(islice(train_likes, 1))

In [ ]:
def get_dict():
    for user, item in train_likes:
        yield {'user_id': user, 'item_id': item}

In [ ]:
vectorizer = sklearn.feature_extraction.DictVectorizer(dtype=np.bool)
user_item = vectorizer.fit_transform(get_dict())

In [ ]:
user_item

# SVD

$\DeclareMathOperator*{\argmin}{arg\,min}$
There are two different ways to look at the truncated SVD of a matrix. One is the standard definition:

> First you do the SVD: $\underset{n\times m}{X} = \underset{n\times n}{U} \overset{n\times m}{\Sigma} \underset{m\times m}{V^T}$, where $U$ and $V$ are rotation matrices, and $\Sigma$ has the singular values along the diagonal. Then you pick the top $k$ singular values, zero out the rest, and hack off irrelevant rows and columns to make a $k$-rank approximation to the original: $X \approx \tilde{X} = \underset{n\times k}{\tilde{U}} \overset{k\times k}{\tilde{\Sigma}} \underset{k\times m}{\tilde{V}^T}$

This is all fine and dandy, but it doesn't make sense when talking about matrices with missing values. However, there's an interesting property of the $k$-truncated SVD--It's the best $k$-rank approximation to the original! That is:

> $ \tilde{X} = \argmin_{B : rank(B)=k} \displaystyle\sum\limits_{i,j} (X_{ij} - B_{ij})^2$

This property seems easy to generalize to the missing value case. Basically you're looking for a $k$-rank matrix that minimizes the element-wise mean squared error across the known entries of the original matrix. That is, when you're training the system, you ignore all of the missing values.

Then, once you've come up with a suitably "close" $k$-rank approximation to the original, you use it to fill in the missing values. That is, if $X_{ij}$ was missing, then you fill in $\tilde{X}_{ij}$. Tada! You are now done.

-- http://stats.stackexchange.com/a/35460

In [ ]:
u, s, v = sp.sparse.linalg.svds(user_item.astype(np.float32), k=6)

In [ ]:
u.shape, s.shape, v.shape

In [ ]:
u

In [ ]:
s

In [ ]:
v

# Predicting

Let the matrix A be such that rows are the users and the columns are the items that the user likes. One way to think of SVD is as follows: SVD finds a hidden feature space where the users and items they like have feature vectors that are closely aligned.

So, when we compute $A = U \times s \times V$, the $U$ matrix represents the feature vectors corresponding to the users in the hidden feature space and the $V$ matrix represents the feature vectors corresponding to the items in the hidden feature space.

Now, if I give you two vectors from the same feature space and ask you to find if they are similar, what is the simplest thing that you can think of for accomplishing that? Dot product.

So, if I want to see user $i$ likes item $j$, all I need to do is take the dot product of the $i$th entry in $U$ and $j$th entry in V. Of course, dot product is by no means the only thing you can apply, any similarity measure that you can think of is applicable.

-- http://stats.stackexchange.com/a/31101

In [ ]:
np.dot(u[:10,], v) # get the probabilities for first ten users -> all films

In [ ]:
np.dot(u[1,:], v[:,546]) # get the probabilities for user 1 -> film 546

### Recommended interface
```python
get_like_likelihood(user_id: str, item_id: str) -> float
```

In [ ]:
@overload
def get_like_likelihood(user_id: int, item_id: int) -> float:
    return np.dot(u[user_id,:], v[:,item_id])

In [ ]:
def get_vectorized_id(mapping: str) -> int:
    return vectorizer.vocabulary_[mapping]

In [ ]:
@overload
def get_like_likelihood(user_id: str, item_id: str) -> float:
    user_id_v = get_vectorized_id('user_id={}'.format(user_id))
    item_id_v = get_vectorized_id('item_id={}'.format(item_id))
    
    return get_like_likelihood(user_id_v, item_id_v)

In [ ]:
get_like_likelihood('70a2805f307f49ec42d4309190daa587', '0ccd2d96eb4b393ecb8cf4e55a6544b6')